In [1]:
!pip install -U "langchain==0.3.*" "langchain-core==0.3.*" "langchain-community==0.3.*" "langgraph==0.3.*" "langchain-huggingface" "huggingface_hub" "sentence-transformers" wikipedia -q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 458.9/458.9 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.2/148.2 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 553.3/553.3 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
def load_api_keys(filepath="api_key.txt"):
    with open(filepath, "r") as f:
        for line in f:
            line = line.strip()
            if line and "=" in line:
                key, value = line.split("=", 1)
                os.environ[key.strip()] = value.strip()

path = '/content/drive/MyDrive/LangGraph/'

# API 키 로드 및 환경변수 설정
load_api_keys(path + 'api_key.txt')

# Server

In [ ]:
pip install langserve

In [ ]:
#!/usr/bin/env python
from fastapi import FastAPI
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langserve import add_routes


app = FastAPI(
  title="LangChain Server",
  version="1.0",
  description="A simple api server using Langchain's Runnable interfaces",
)

add_routes(
    app,
    ChatOpenAI(),
    path="/openai",
)

model = ChatOpenAI()
prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
add_routes(
    app,
    prompt | model,
    path="/joke",
)

if __name__ == "__main__":
    import uvicorn

    uvicorn.run(app, host="localhost", port=8000)

# sdk client

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableMap
from langserve import RemoteRunnable

openai = RemoteRunnable("http://localhost:8000/openai/")
joke_chain = RemoteRunnable("http://localhost:8000/joke/")

print(joke_chain.invoke({"topic": "parrots"}))
"""
content="Sure, here's a joke for you:\n\nWhy don't scientists trust parrots?\n\nBecause they always give them quack advice!"
"""

prompt = ChatPromptTemplate.from_messages(
    [("system", "Tell me a long story about {topic}")]
)

# Can define custom chains
chain = prompt | RunnableMap({
    "openai": openai
})

print(chain.batch([{ "topic": "parrots" }]))
"""
[{'openai': AIMessage(content='Once upon a time, in a lush tropical rainforest, there lived a colorful and vibrant community of parrots. These parrots were renowned for their intelligence, beauty, and lively personalities. They were the guardians of the forest, spreading joy and laughter with their melodious songs and playful behavior.\n\nAt the heart of the parrot community was a wise and respected elder named Ollie. Ollie had a rich plumage of emerald green feathers, with touches of bright red and blue. His eyes twinkled with an ancient wisdom that had been passed down through generations.\n\nOne day, as the parrots gathered around Ollie, he shared a tale from long ago. It was a story about an ancient treasure hidden deep within the forest. This treasure, known as the "Jewel of the Rainforest," was said to possess magical powers that could bring harmony and balance to all living creatures.\n\nThe parrots were intrigued by the tale and decided to embark on a quest to find the Jewel of the Rainforest. They believed that by harnessing its powers, they could protect their home from any harm that might come its way.\n\nLed by Ollie, the parrots set off on their adventure, flying through the dense foliage, and singing songs of courage and determination. Along the way, they encountered various challenges, but their unity and unwavering spirit kept them going.\n\nAs they ventured deeper into the forest, they faced treacherous rivers, towering trees, and hidden predators. However, their sharp minds and agile wings helped them overcome every obstacle. They supported one another, sharing their knowledge and strength.\n\nDuring their journey, the parrots discovered the incredible diversity of the rainforest. They marveled at the exotic flowers, cascading waterfalls, and the symphony of sounds that filled the air. They also encountered other creatures, like mischievous monkeys swinging from the trees and wise old turtles slowly making their way across the forest floor.\n\nOne day, after months of relentless searching, the parrots stumbled upon a hidden cave. Inside, they found an ethereal glow emanating from a magnificent jewel. It was the Jewel of the Rainforest, shimmering with every color imaginable. The parrots knew they had found what they were looking for.\n\nAs they gathered around the jewel, a magical energy enveloped them. They felt a sense of peace and enlightenment wash over them, as if the forest itself had embraced them. From that moment on, the parrots became the true guardians of the rainforest, using the Jewel\'s powers to protect and preserve their home.\n\nWith the Jewel\'s magic, the parrots brought harmony to the ecosystem, ensuring that all creatures lived in balance and peace. They sang their songs of unity, healing the wounds of the forest and spreading happiness throughout.\n\nWord of the parrots\' noble efforts spread far and wide, attracting visitors from across the world who wanted to witness the wonders of the rainforest. People marveled at the parrots\' intelligence, their ability to mimic human speech, and their vibrant colors.\n\nThe parrots became ambassadors of the rainforest, teaching humans the importance of conservation and the need to protect the Earth\'s natural wonders. Their enchanting presence brought joy to all who encountered them, reminding everyone of the beauty and magic that resided within nature.\n\nAnd so, the parrots lived on, their legacy carried through the generations. They continued to guard the rainforest, ensuring its prosperity and safeguarding the Jewel of the Rainforest. Their story became a legend, passed down through time, reminding everyone of the power of unity, wisdom, and the magic that lies within the hearts of parrots.')}]
"""

# client

In [ ]:
import json
import requests
"""
POST /my_runnable/invoke - invoke the runnable on a single input
POST /my_runnable/batch - invoke the runnable on a batch of inputs
POST /my_runnable/stream - invoke on a single input and stream the output
POST /my_runnable/stream_log - invoke on a single input and stream the output, including output of intermediate steps as it's generated
GET /my_runnable/input_schema - json schema for input to the runnable
GET /my_runnable/output_schema - json schema for output of the runnable
GET /my_runnable/config_schema - json schema for config of the runnable
"""
response = requests.post(
    "http://localhost:8000/joke/invoke",
    json={'input': {'topic': 'cats'}}
)
print(response.json())
"""
{
    'output': {
        'content': "Sure, here's a cat-related joke for you:\n\nWhy don't cats play poker in the wild?\n\nToo many cheetahs!",
        'additional_kwargs': {},
        'type': 'ai',
        'example': False
    },
    'callback_events': [],
    'metadata': {
        'run_id': '8ed5fa43-6e1f-4743-8310-77dbe52503c0'}}
"""

response = requests.get(
    "http://localhost:8000/joke/input_schema",
)
print(response.json())
"""
{'title': 'PromptInput', 'type': 'object', 'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}
"""

response = requests.post(
    "http://localhost:8000/joke/stream",
    json={'input': {'topic': 'cats'}},
    stream=True
)

# Check if the request was successful
if response.status_code == 200:
    try:
        current_event = None
        event_data = []
        for line in response.iter_lines():
            if line:  # filter out keep-alive new lines
                # print(line.decode('utf-8'))
                line = line.decode('utf-8')
                """
                event: data
                data: {"content":"","additional_kwargs":{},"type":"AIMessageChunk","example":false}
                event: data
                data: {"content":"Sure","additional_kwargs":{},"type":"AIMessageChunk","example":false}
                event: data
                ...
                """
                if line.startswith("event:"):
                    current_event = line.split(":", 1)[1].strip()
                elif line.startswith("data:"):
                    data = line.split(":", 1)[1].strip()
                    try:
                        parsed_data = json.loads(data)
                        """
                        {'run_id': '9d77c51b-6e0d-457f-a75e-0961f205cc8f'}
                        {'content': '', 'additional_kwargs': {}, 'type': 'AIMessageChunk', 'example': False}
                        {'content': 'Sure', 'additional_kwargs': {}, 'type': 'AIMessageChunk', 'example': False}
                        {'content': ',', 'additional_kwargs': {}, 'type': 'AIMessageChunk', 'example': False}
                        ...
                        """
                        event_data.append((current_event, parsed_data))

                    except json.JSONDecodeError:
                        print(f"Error decoding JSON: {data}")
    except KeyboardInterrupt:
        print("Streaming stopped by user.")
else:
    print(f"Failed to stream data: {response.status_code}")